In [1]:
import numpy as np
from numpy import pi, cos, sin, exp, log, arctan, absolute
import matplotlib.pyplot as plt
import imageio

import astropy.coordinates as coord
import astropy.units as u

In [2]:
class Detector:
    def __init__(self, longitude, latitude):
        self.lon = float(longitude)
        self.lat = float(latitude)

def rotate(angles, alpha, beta):
    """
    Rotation in spherical coordinates.
    Firt rotate along z axis, than rotate along y' axis.
    See https://stla.github.io/stlapblog/posts/RotationSphericalCoordinates.html for detail.
    
    Parameters
    ----------
    angles : numpy arrays
        The numpy array of [theta, phi]
    alpha : float
        The rotation angle along z axis
    beta : float 
        The rotation angle along y axis
    """
    
    # the tranform matrix
    tfMatrixZ = np.array([[exp(-1j*alpha/2), 0], [0, exp(1j*alpha/2)]])
    tfMatrixY = np.array([[cos(beta/2), -sin(beta/2)], [sin(beta/2), cos(beta/2)]])
    tfMatrix = np.dot(tfMatrixY, tfMatrixZ)
    
    # return data initialize
    length = len(angles)
    anglesNew = np.zeros([length, 2])
    
    for i in range(length):
        angle = angles[i]
        comp = np.array([cos(angle[0]/2), exp(1j*angle[1]) * sin(angle[0]/2)])
        compNew = np.dot(comp, tfMatrix)
        compRatio = compNew[1] / compNew[0]
        angleNew = np.array([2*arctan(absolute(compRatio)), np.angle(compRatio)])
        anglesNew[i] = angleNew.copy()
    return anglesNew
        

In [3]:
for i in range(24):
    """
    intialize figure
    """
    fig = plt.figure(figsize=(8,6), dpi=250)
    ax = fig.add_subplot(111, projection="mollweide")


    """
    intialize data
    """

    # IceCube data
    icecube = Detector(0, -pi/2)
    icecube.points = np.array([[theta, phi] 
                               for theta in np.arange(60/180*pi, 95/180*pi, 0.05) 
                               for phi in np.arange(-pi, pi, 0.05)])
    # SCS data
    lonEvo = 112.3/180*pi + i*pi/12
    if lonEvo > pi:
        lonEvo -= 2*pi
    scs = Detector(lonEvo, 16.8/180*pi)
    scs.points = rotate(icecube.points, scs.lon, (pi/2+scs.lat))


    """
    plot and config the axes
    """

    # plot IceCube data
    ax.fill_between([-pi, pi], [-pi/36, -pi/36], [pi/6, pi/6], alpha=.3, color='royalblue')
    # ax.scatter(icecube.points[:,1], pi/2-icecube.points[:,0], alpha=.7, s=0.15, color='royalblue')
    ax.scatter(icecube.lon, icecube.lat+0.1, s=2., marker='x', color='royalblue')
    font = {'family': 'serif', 'weight': 'normal', 'size': 13}
    ax.text(icecube.lon+0.07, icecube.lat+0.17, 'IceCube', color='royalblue', fontdict=font)

    # plot scs data
    ax.scatter(scs.points[:,1], pi/2-scs.points[:,0], alpha=.7, s=0.15, color='darkred')
    ax.scatter(scs.lon, scs.lat, s=2., marker='x', color='darkred')
    ax.text(scs.lon+0.07, scs.lat+0.07, 'South China Sea', color='darkred', fontdict=font)

    # config the axes
    ax.set_xticks([-pi*2/3, -pi/3, 0, pi/3, pi*2/3, pi])
    ax.set_xticklabels(['16h', '20h', '0h', '4h', '8h', '12h'])
    ax.set_yticks([-pi/3, -pi/6, 0, pi/6, pi/3])
    ax.set_yticklabels([r'$-60^\circ$', r'$-30^\circ$', r'$0^\circ$', r'$30^\circ$', r'$60^\circ$'])
    ax.grid(True)
    plt.savefig('./figs/testFig'+str(i)+'.png')
    fig.clear()
    plt.close()

In [16]:
# generate gif
filenames = ['./figs/testFig'+str(i)+'.png' for i in range(24)]
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./figs/testGif.gif', images, duration=0.25)
